In [1]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

c:\Users\Murtuza\anaconda3\envs\MLenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#http://images.cocodataset.org/val2017/000000039769.jpg
url = "https://miro.medium.com/v2/resize:fit:1400/1*EYFejGUjvjPcc4PZTwoufw.jpeg"
image = Image.open(requests.get(url, stream=True).raw)

In [4]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

c:\Users\Murtuza\anaconda3\envs\MLenv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Murtuza\.cache\huggingface\hub\models--facebook--detr-resnet-50. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [10]:
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

In [11]:
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

Detected bicycle with confidence 0.998 at location [201.46, 316.75, 275.05, 440.61]
Detected car with confidence 0.972 at location [78.15, 224.78, 111.76, 301.76]
Detected car with confidence 0.999 at location [557.24, 205.73, 807.89, 360.62]
Detected truck with confidence 0.995 at location [99.32, 89.36, 257.0, 312.53]
Detected potted plant with confidence 0.964 at location [182.27, 230.89, 284.15, 324.68]
Detected car with confidence 0.986 at location [751.35, 199.41, 871.9, 346.56]
Detected truck with confidence 0.977 at location [0.08, 90.52, 97.64, 356.89]
Detected car with confidence 0.973 at location [419.45, 225.71, 471.58, 267.25]
Detected person with confidence 0.998 at location [157.93, 145.76, 299.6, 400.61]
Detected car with confidence 0.999 at location [286.25, 212.36, 460.44, 352.13]
Detected car with confidence 0.969 at location [422.7, 228.23, 477.44, 318.9]
Detected car with confidence 0.999 at location [464.27, 216.68, 597.65, 338.15]


In [14]:
from PIL import Image

In [15]:
for box in results["boxes"]:
    box = [round(i, 2) for i in box.tolist()]
    cropped_object = image.crop(box)
    
    cropped_object.show()